# Evaluate The Models Performance, French Language

In [1]:
import numpy as np

from silence_tensorflow import silence_tensorflow
silence_tensorflow()  # silence TF warnings
import tensorflow as tf

from utils import loadFile
from dataProcessing import encodeData, insertTarget, processingScriber, processingOPUS
from transformers import AutoTokenizer
from transformers import TFCamembertModel, TFCamembertForMaskedLM
from datetime import datetime
import os
import json
import sys

In [2]:
### Path To Models
# path = "ModelsExpScriber/20200604_163315/"
# path = "Models/20200530_161559/"  # Baseline Model
path = "ModelsExpScriber/20200827_113149/"
# path = "ModelsExpScriber/20200826_090912/"

In [3]:
### instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained("jplu/tf-camembert-base", do_lower_case=True)

In [4]:
### puntuation encoder
punctuationEnc = {
    'SPACE': 0,
    'PERIOD': 1,
}

## Hyper-parameters

In [5]:
vocabSize = 32005
sequenceSize = 32
batchSize = 32

## Get Dataset

In [6]:
# this is the file structured in sentences
dataName = "./DataScriber/raw.processed.Test_01.txt"

# from sentences to columns words+punctuation
dataSet = loadFile(processingScriber(dataName))

### Encode Data and insert target
X, y = encodeData(dataSet, tokenizer, punctuationEnc)

### Create Sequences With The Target
XMod = insertTarget(X, sequenceSize)

# one hot encode the labels
yMod = tf.one_hot(y, len(punctuationEnc), dtype='int64').numpy()

# build the datasets
dataset = tf.data.Dataset.from_tensor_slices((XMod, yMod)).batch(batchSize)

print("\nTraining Dataset Tensor Shape = ", XMod.shape)


Training Dataset Tensor Shape =  (255, 32)


In [7]:
# ### Get Percentage Of Ones In The Dataset

# indTup = np.where(y==1)
# ind = indTup[1]
# print(np.sum(ind))

## Build Baseline Model

In [8]:
bert_input = tf.keras.Input(shape=(sequenceSize), dtype='int32', name='bert_input')
x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(bert_input)[0]
x = tf.keras.layers.Reshape((sequenceSize*vocab_size,))(x)
dense_out = tf.keras.layers.Dense(len(punctuation_enc), activation='softmax')(x)
model = tf.keras.Model(bert_input, dense_out, name='model')

## Build Model, No LM head

In [9]:
# hiddenDimension = 768
# bert_input = tf.keras.Input(shape=(sequenceSize), dtype='int32', name='bert_input')
# x = TFCamembertModel.from_pretrained("jplu/tf-camembert-base")(bert_input)[0]
# x = tf.keras.layers.Reshape((sequenceSize*hiddenDimension,))(x)
# dense_out = tf.keras.layers.Dense(len(punctuationEnc), activation='softmax')(x)
# model = tf.keras.Model(bert_input, dense_out)

Some weights of the model checkpoint at jplu/tf-camembert-base were not used when initializing TFCamembertModel: ['lm_head']
- This IS expected if you are initializing TFCamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFCamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFCamembertModel were initialized from the model checkpoint at jplu/tf-camembert-base.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFCamembertModel for predictions without further training.


## Build Model, Use LM head, Plus Two Fully Connwcted Layers

In [10]:
# bert_input = tf.keras.Input(shape=(sequenceSize), dtype='int32', name='bert_input')
# x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(bert_input)[0]
# x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
# x = tf.keras.layers.Dense(64, activation='relu')(x)
# dense_out = tf.keras.layers.Dense(len(punctuationEnc))(x)
# model = tf.keras.Model(bert_input, dense_out, name='model')

## Compile Model

In [11]:
model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[tf.keras.metrics.Recall(class_id=0, name='Rec_0'),
                       tf.keras.metrics.Precision(class_id=0, name='Prec_0'),
                       tf.keras.metrics.Recall(class_id=1, name='Rec_1'),
                       tf.keras.metrics.Precision(class_id=1, name='Prec_1'),
                      ])

## Evaluate the Models

In [12]:
### Get List of the Models in the Output Folder

modelsLst = []
for r, d, f in os.walk(path):
    for file in sorted(f):
        if ".index" in file:
            modelsLst.append(file[:-6])

In [13]:
### Compute F1 Score

def compF1(rec, pre):
    if pre + rec == .0:
        return .0
    else:
        return 2 * (pre*rec) / (pre+rec)

In [14]:
### Evaluate the Models

for i in range(len(modelsLst)):
    # get the path
    checkpointPath = path + modelsLst[i]
    print(checkpointPath)

    # load the weights
    model.load_weights(checkpointPath)

    evaluation = model.evaluate(dataset)
    f1_0 = compF1(evaluation[1],evaluation[2])
    f1_1 = compF1(evaluation[3],evaluation[4])
    print("F1_0 = {:11.7f}     F1_1 = {:11.7f}".format(f1_0, f1_1))

ModelsExpScriber/20200827_113149/cp-001.ckpt
8/8 [==============================] - 11s 1s/step - loss: 0.3855 - Rec_0: 1.0000 - Prec_0: 0.9725 - Rec_1: 0.0000e+00 - Prec_1: 0.0000e+00
F1_0 =   0.9860835     F1_1 =   0.0000000
ModelsExpScriber/20200827_113149/cp-002.ckpt
8/8 [==============================] - 11s 1s/step - loss: 0.1996 - Rec_0: 1.0000 - Prec_0: 0.9725 - Rec_1: 0.0000e+00 - Prec_1: 0.0000e+00
F1_0 =   0.9860835     F1_1 =   0.0000000
ModelsExpScriber/20200827_113149/cp-003.ckpt
8/8 [==============================] - 11s 1s/step - loss: 0.1380 - Rec_0: 1.0000 - Prec_0: 0.9725 - Rec_1: 0.0000e+00 - Prec_1: 0.0000e+00
F1_0 =   0.9860835     F1_1 =   0.0000000
